Getting the data from the repository...don't run unless you don't have the data!

!apt-get -y install curl

!curl -o BioMedSent/BioMedSentences.tar.zip http://i.stanford.edu/hazy/opendata/bmc/bmc_full_dddb_20150927_9651bf4a468cefcea30911050c2ca6db.tar.bzip2


In [1]:
import string
import os
import pickle
from collections import defaultdict

from nltk.corpus import stopwords

class Sentence:
    def __init__(self, document, sentenceNumber, wordList, lemmaList, posList):
        self.document = document
        self.sentenceNumber = sentenceNumber
        self.wordList = wordList
        self.lemmaList = lemmaList
        self.posList = posList
        self.sentence = " ".join([word for word in wordList if word not in string.punctuation])
        self.lemmaSent = " ".join([word for word in lemmaList if word not in string.punctuation])

fileList = os.listdir("../BioMedSent/bmc_full_dddb")

sentList = []

fileList.sort()

for n in range(1,5):
    f = open("../BioMedSent/bmc_full_dddb/" + fileList[n], 'r')
    for line in f:
        sentList.append(line.split('\t'))
    
len(sentList)

2064735

In [2]:
#sentList[7]

docList = []

for thing in sentList:
    docList.append(thing[0])

len(set(docList))

8756

In [3]:
sentObjList = []

for item in sentList:
    wordList = item[3].replace('"',"").replace("{","").replace("}","").split(",")
    wordList = filter(None, wordList)
    
    posList = item[4].replace('"',"").replace("{","").replace("}","").split(",")
    
    lemmaList = item[6].replace('"',"").replace("{","").replace("}","").split(",")
    lemmaList = filter(None, lemmaList)
    
    sentObjList.append(Sentence(item[0], item[1], wordList, lemmaList, posList))
    
#len(sentList)
sentObjList[7].lemmaSent

'to this end we have develop a web server tool TarFisDock target fishing docking which have be use widely by other'

In [4]:
headingsDict = defaultdict(dict)

for sent in sentObjList:
    if len(sent.wordList) == 1:
        #print(sent.wordList)
        word = string.upper(sent.wordList[0]).strip()
        if word == 'INTRODUCTION' or word == 'BACKGROUND':
            headingsDict[sent.document]["introduction"] = sent.sentenceNumber
        elif word == 'METHODS':
            headingsDict[sent.document]["methods"] = sent.sentenceNumber
        elif word == 'RESULTS':
            headingsDict[sent.document]["results"] = sent.sentenceNumber
        elif word == 'DISCUSSION':
            headingsDict[sent.document]["discussion"] = sent.sentenceNumber
        elif word == 'CONCLUSION':
            headingsDict[sent.document]["conclusion"] = sent.sentenceNumber
        elif word == 'REFERENCES':
            headingsDict[sent.document]["references"] = sent.sentenceNumber
        

headingsDict.keys()

['BMC_Public_Health_2009_May_28_9_162.nxml.txt.nlp',
 'BMC_Res_Notes_2009_Mar_10_2_39.nxml.txt.nlp',
 'BMC_Med_Ethics_2009_Jul_6_10_9.nxml.txt.nlp',
 'BMC_Pharmacol_Toxicol_2013_Sep_28_14_48.nxml.txt.nlp',
 'BMC_Cancer_2006_Nov_8_6_263.nxml.txt.nlp',
 'BMC_Vet_Res_2009_Feb_4_5_7.nxml.txt.nlp',
 'BMC_Cancer_2014_Nov_6_14(1)_814.nxml.txt.nlp',
 'BMC_Res_Notes_2013_Nov_12_6_455.nxml.txt.nlp',
 'BMC_Evol_Biol_2011_Jul_1_11_188.nxml.txt.nlp',
 'BMC_Public_Health_2013_Apr_12_13_338.nxml.txt.nlp',
 'BMC_Biotechnol_2006_Jan_16_6_6.nxml.txt.nlp',
 'BMC_Health_Serv_Res_2009_Sep_15_9_163.nxml.txt.nlp',
 'BMC_Med_Res_Methodol_2013_Sep_16_13_114.nxml.txt.nlp',
 'BMC_Genomics_2014_Oct_3_15(1)_846.nxml.txt.nlp',
 'BMC_Med_2007_Jan_1_5_1.nxml.txt.nlp',
 'BMC_Plant_Biol_2012_Feb_14_12_21.nxml.txt.nlp',
 'BMC_Infect_Dis_2014_May_27_14(Suppl_3)_O13.nxml.txt.nlp',
 'BMC_Nephrol_2014_Apr_4_15_58.nxml.txt.nlp',
 'BMC_Complement_Altern_Med_2014_Feb_25_14_77.nxml.txt.nlp',
 'BMC_Geriatr_2014_Jan_10_14_3.nxml.

In [5]:
documentDict = defaultdict(list)
docPartsDict = defaultdict(lambda : defaultdict(list))
docPartsCombinedDict = defaultdict(dict)

for item in sentObjList:
    documentDict[item.document].append(item)
    
for document in documentDict.keys():
    docSentList = documentDict[document]
    introNum = int(headingsDict[document].get("introduction", -1))
    methoNum = int(headingsDict[document].get("methods", -1))
    resultNum = int(headingsDict[document].get("results", -1))
    discussNum = int(headingsDict[document].get("discussion", -1))
    conclusionNum = int(headingsDict[document].get("conclusion", -1))
    refNum = int(headingsDict[document].get("references", -1))

    for sent in docSentList:
        label = "noSection"
        dist = int(sent.sentenceNumber)
        sentNumber = int(sent.sentenceNumber)
        
        if dist > sentNumber - introNum and sentNumber - introNum > 0:
            label = "introduction"
            dist = sentNumber - introNum
        if dist > sentNumber - methoNum and sentNumber - methoNum > 0:
            label = "methods"
            dist = sentNumber - methoNum
        if dist > sentNumber - resultNum and sentNumber - resultNum > 0:
            label = "results"
            dist = sentNumber - resultNum
        if dist > sentNumber - discussNum and sentNumber - discussNum > 0:
            label = "discussion"
            dist = sentNumber - discussNum
        if dist > sentNumber - conclusionNum and sentNumber - conclusionNum > 0:
            label = "conclusion"
            dist = sentNumber - conclusionNum
        if dist > sentNumber - refNum and sentNumber - refNum > 0:
            label = "references"
            dist = sentNumber - refNum
        
        docPartsDict[document][label].append(sent)
    
    for x in docPartsDict[document].keys():
        docPartsCombinedDict[document][x] = " ".join(y.sentence for y in sorted(docPartsDict[document][x], key=lambda z: z.sentenceNumber))
        docPartsCombinedDict[document][x] = " ".join([word for word in docPartsCombinedDict[document][x].split() if word not in (stopwords.words('english'))])
        
#print(docPartsDict.keys())
#print(docPartsCombinedDict["AAPS_J_2011_Dec_22_14(1)_68-78.nxml.txt.nlp"].keys())  

LookupError: 
**********************************************************************
  Resource u'corpora/stopwords' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [ ]:
validDocsDict = defaultdict(dict)

for doc in docPartsCombinedDict.keys():
    tempKeys = docPartsCombinedDict[doc].keys()
    if 'introduction' in tempKeys and 'discussion' in tempKeys and 'conclusion' in tempKeys:
        validDocsDict[doc] = docPartsCombinedDict[doc]

print(str(len(docPartsCombinedDict.keys())))
print(str(len(validDocsDict.keys())))

In [ ]:
import lda
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

documents = []
for k in validDocsDict.keys():
    for j in validDocsDict[k].keys():
        documents.append(validDocsDict[k][j])

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
inArray = tf_vectorizer.fit_transform(documents)

model = lda.LDA(n_topics=6, n_iter=500, random_state=1)
model.fit(X)
topic_word = model.topic_word_
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))